In [2]:
#from group_lasso import LogisticGroupLasso
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import sklearn as sk
from sklearn.metrics import roc_curve, auc
from sklearn import metrics
from sklearn import impute
from sklearn import preprocessing
import itertools
import time

In [59]:
data = pd.read_csv("owid-covid-data-1.csv")
data.shape

(166326, 67)

In [6]:
# descriptive stats

data.describe()

,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
count,1.632930e+05,1.631330e+05,1.611500e+05,1.454510e+05,145487.000000,143390.000000,162535.000000,162375.000000,160398.000000,144706.000000,...,106050.000000,104595.000000,68569.000000,123664.000000,155268.000000,136253.000000,5.696000e+03,5696.000000,5696.000000,5696.000000
mean,2.536044e+06,1.157084e+04,1.156560e+04,5.766407e+04,171.137304,172.673031,29447.756785,166.431538,165.507110,509.384956,...,10.627229,32.778221,50.788710,3.027816,73.576309,0.725587,3.761302e+04,9.404336,15.967077,972.197816
std,1.543441e+07,8.442598e+04,8.257830e+04,3.021145e+05,832.251328,817.024076,51852.379656,683.021740,532.174029,784.551311,...,10.558306,13.523688,31.811788,2.450110,7.491615,0.149964,1.043065e+05,16.439173,30.092830,1420.342295
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000,0.000000,0.001000,0.000000,0.000000,0.000000,...,0.100000,7.700000,1.188000,0.100000,53.280000,0.394000,-3.772610e+04,-28.450000,-95.920000,-1826.595723
25%,2.001000e+03,1.000000e+00,7.000000e+00,7.900000e+01,0.000000,0.143000,623.579000,0.042000,1.630000,18.580250,...,1.900000,21.600000,19.351000,1.300000,69.500000,0.602000,-7.515000e+01,-0.722500,-0.752500,-29.788442
50%,2.611700e+04,7.900000e+01,1.071430e+02,7.830000e+02,2.000000,2.429000,4731.521000,11.439000,18.829000,127.737500,...,6.300000,31.400000,49.839000,2.400000,75.050000,0.743000,3.424600e+03,6.065000,7.195000,473.393182
75%,2.987020e+05,1.063000e+03,1.146000e+03,7.307000e+03,20.000000,21.286000,37724.466000,101.289500,120.859750,711.954750,...,19.300000,41.300000,83.241000,4.000000,78.930000,0.845000,2.478462e+04,14.520000,22.997500,1656.361067
max,4.451295e+08,4.206334e+06,3.444237e+06,5.995245e+06,18020.000000,14689.143000,706541.904000,51427.491000,16052.608000,6322.263000,...,44.000000,78.100000,100.000000,13.800000,86.750000,0.957000,1.080748e+06,111.010000,374.930000,9153.060433


In [52]:
# missingness

missing = pd.DataFrame(data.isnull().sum())
missing['% missing'] = data.isnull().sum()/data.shape[0]
missing.columns = ["Missing Count", "Proportion Missing"]
missing = missing.sort_values(by="Proportion Missing", ascending=True)

In [53]:
missing = missing[missing["Proportion Missing"] < .20]
missing

,Missing Count,Proportion Missing
iso_code,0,0.000000
location,0,0.000000
date,0,0.000000
population,1075,0.006463
total_cases,3033,0.018235
new_cases,3193,0.019197
total_cases_per_million,3791,0.022793
new_cases_per_million,3951,0.023755
new_cases_smoothed,5176,0.031120
new_cases_smoothed_per_million,5928,0.035641


In [30]:
acceptable_columns = list(missing.index.values)
acceptable_columns

['iso_code',
 'location',
 'date',
 'population',
 'total_cases',
 'new_cases',
 'total_cases_per_million',
 'new_cases_per_million',
 'new_cases_smoothed',
 'new_cases_smoothed_per_million',
 'continent',
 'life_expectancy',
 'population_density',
 'new_deaths',
 'total_deaths',
 'new_deaths_per_million',
 'total_deaths_per_million',
 'diabetes_prevalence',
 'new_deaths_smoothed',
 'new_deaths_smoothed_per_million',
 'gdp_per_capita',
 'median_age',
 'aged_70_older',
 'cardiovasc_death_rate',
 'aged_65_older',
 'human_development_index']

In [33]:
continent_data = data.groupby("location").count()
continent_data = continent_data.drop(columns=[col for col in continent_data if col not in acceptable_columns])
continent_data

,iso_code,continent,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,...,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,cardiovasc_death_rate,diabetes_prevalence,life_expectancy,human_development_index
location,,,,,,,,,,,,,,,,,,,,,
Afghanistan,741,741,741,741,739,721,713,713,707,741,...,741,741,741,741,741,741,741,741,741,741
Africa,752,0,752,751,752,746,728,752,746,751,...,752,0,0,0,0,0,0,0,0,0
Albania,740,740,740,727,726,720,725,725,719,727,...,740,740,740,740,740,740,740,740,740,740
Algeria,740,740,740,740,740,734,724,724,718,740,...,740,740,740,740,740,740,740,740,740,740
Andorra,734,734,734,734,734,728,714,714,708,734,...,734,734,0,0,0,0,734,734,734,734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wallis and Futuna,503,503,503,503,503,497,350,350,344,503,...,503,0,0,0,0,0,0,0,503,0
World,774,0,774,774,774,768,774,774,768,774,...,774,774,774,774,774,774,774,774,774,774
Yemen,695,695,695,695,694,682,675,675,669,695,...,695,695,695,695,695,695,695,695,695,695


In [43]:
missing2 = pd.DataFrame(continent_data.sum(axis=1))
missing2['% missing'] = 1 - (missing2/19350)
missing2 = missing2[missing2["% missing"] < .20]
missing2

,0,% missing
location,,
Afghanistan,18301,0.054212
Albania,18304,0.054057
Algeria,18380,0.050129
Angola,17806,0.079793
Antigua and Barbuda,17869,0.076537
...,...,...
Vietnam,18133,0.062894
World,18552,0.041240
Yemen,17215,0.110336


In [47]:
acceptable_locations = list(missing2.index.values)
acceptable_locations

['Afghanistan',
 'Albania',
 'Algeria',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape Verde',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Curacao',
 'Cyprus',
 'Czechia',
 'Democratic Republic of Congo',
 'Denmark',
 'Djibouti',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Grenada',
 'Guatemala',
 'Guinea',
 'Guinea-Bissau',
 'Guyana',
 'Haiti',
 'Honduras',
 'Hong Kong'

In [60]:
new_data = data[data['location'].isin(acceptable_locations)]
new_data = new_data.drop(columns=[col for col in new_data if col not in acceptable_columns])
new_data

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,cardiovasc_death_rate,diabetes_prevalence,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,39835428.0,54.422,18.6,2.581,1.337,1803.987,597.029,9.59,64.83,0.511
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,39835428.0,54.422,18.6,2.581,1.337,1803.987,597.029,9.59,64.83,0.511
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,39835428.0,54.422,18.6,2.581,1.337,1803.987,597.029,9.59,64.83,0.511
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,39835428.0,54.422,18.6,2.581,1.337,1803.987,597.029,9.59,64.83,0.511
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,39835428.0,54.422,18.6,2.581,1.337,1803.987,597.029,9.59,64.83,0.511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166321,ZWE,Africa,Zimbabwe,2022-03-01,236871.0,491.0,413.000,5395.0,0.0,1.000,...,15092171.0,42.729,19.6,2.822,1.882,1899.775,307.846,1.82,61.49,0.571
166322,ZWE,Africa,Zimbabwe,2022-03-02,237503.0,632.0,416.286,5396.0,1.0,1.143,...,15092171.0,42.729,19.6,2.822,1.882,1899.775,307.846,1.82,61.49,0.571
166323,ZWE,Africa,Zimbabwe,2022-03-03,237503.0,0.0,362.286,5396.0,0.0,0.857,...,15092171.0,42.729,19.6,2.822,1.882,1899.775,307.846,1.82,61.49,0.571
166324,ZWE,Africa,Zimbabwe,2022-03-04,238739.0,1236.0,467.429,5397.0,1.0,0.714,...,15092171.0,42.729,19.6,2.822,1.882,1899.775,307.846,1.82,61.49,0.571
